# Setup and Imports

Import useful libraries for defining our model, set up the model

In [1]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.namespaces import BRICK
from buildingmotif.dataclasses import Model, Library

In [2]:
bm = BuildingMOTIF("sqlite://") # in-memory

In [3]:
BLDG = Namespace("urn:my_site/")
model = Model.create(BLDG)

In [4]:
constraints = Library.load(ontology_graph="constraints/constraints.ttl")
brick = Library.load(ontology_graph="../libraries/brick/Brick-subset.ttl")
g36 = Library.load(directory="../libraries/ashrae/guideline36")

- simulate looking at a mechanical schedule:
    - see number and type of equipment
    - **our example**: 1 AHU, 2 RVAVs
- now look at libraries to see if there are some useful "shapes"
    - based on forecasted app requirements
    - use these to inform model creation

In [5]:
sc = g36.get_shape_collection()

In [6]:
sc.get_shapes_about_class(BRICK.AHU)

[]

In [7]:
sc.get_shapes_about_class(BRICK.RVAV)

[rdflib.term.URIRef('urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat')]

# Write site manifest

- write a "manifest" to encode our requirements:
    - 1 AHU
    - 2 RVAVs; should run G36 sequence of operations
- this is the *low level* view -- will be abstracted by us and other developers

In [8]:
%%bash
cat <<EOF > my_site.ttl
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix constraint: <https://nrel.gov/BuildingMOTIF/constraints#> .
@prefix : <urn:my_site_constraints/> .

: a owl:Ontology ;
    owl:imports <https://brickschema.org/schema/1.3/Brick> ;
    owl:imports <https://nrel.gov/BuildingMOTIF/constraints> ;
    owl:imports <urn:ashrae/g36> .

# read off of mechanical schedule
:vav-count a sh:NodeShape ;
    sh:message "Need 2 VAVs" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 2 ;
    constraint:class <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .

:ahu-count a sh:NodeShape ;
    sh:message "Need 1 AHU" ;
    sh:targetNode : ; # target *this* graph
    constraint:exactCount 1 ;
    constraint:class brick:AHU .
EOF

Now we load our manifest into BuildingMOTIF

In [9]:
manifest = Library.load(ontology_graph="my_site.ttl")

# Iterative model creation

- validate model to determine if it is *semantically sufficient*
- if this fails, BuildingMOTIF will use resulting report to help you patch the model

In [10]:
# pass our collections of shapes in to validate the model
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), 
                      g36.get_shape_collection(), brick.get_shape_collection()])
print(f"Model is valid? {ctx.valid}")
# higher-level descriptions of each of the violations
print("Reasons why:")
for diff in ctx.diffset:
    print(" -" + diff.reason())

Model is valid? False
Reasons why:
 -Graph did not have 1 instances of https://brickschema.org/schema/Brick#AHU
 -Graph did not have 2 instances of urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat


- some constraints can be interpreted by BuildingMOTIF
- BuildingMOTIF identifies common patterns
    - presents these as new templates to the user
    - templates provide structure to input from user/other software

In [11]:
generated_templates = ctx.as_templates()
for t in generated_templates:
    print('-'*80)
    print(t.body.serialize())
    ident = input("Give value for 'name' in the above template: ")
    model.add_graph(t.evaluate({"name": BLDG[ident]}))

--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

P:name a brick:AHU .




Give value for 'name' in the above template:  ahu


--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .

P:name a <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .




Give value for 'name' in the above template:  vav1


--------------------------------------------------------------------------------
@prefix P: <urn:___param___#> .

P:name a <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .




Give value for 'name' in the above template:  vav2


In [12]:
# now we can look at our model ( so far )
print(model.graph.serialize())


<urn:my_site/> a <http://www.w3.org/2002/07/owl#Ontology> .

<urn:my_site/ahu> a <https://brickschema.org/schema/Brick#AHU> .

<urn:my_site/vav1> a <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .

<urn:my_site/vav2> a <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> .




In [13]:
# validate it again!
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()])
print(f"Model is valid? {ctx.valid}")
print("Reasons why:")
for diff in ctx.diffset:
    print(" -" + diff.reason())

Model is valid? False
Reasons why:
 -urn:my_site/vav1 needs between 1 and None instances of https://brickschema.org/schema/Brick#Discharge_Air_Flow_Sensor on path https://brickschema.org/schema/Brick#hasPoint
 -urn:my_site/vav2 needs between 1 and None instances of https://brickschema.org/schema/Brick#Discharge_Air_Flow_Sensor on path https://brickschema.org/schema/Brick#hasPoint
 -urn:my_site/vav2 needs to be a https://brickschema.org/schema/Brick#VAV
 -urn:my_site/vav1 needs to be a https://brickschema.org/schema/Brick#VAV
 -urn:my_site/vav2 needs to be a https://brickschema.org/schema/Brick#RVAV
 -urn:my_site/vav1 needs to be a https://brickschema.org/schema/Brick#RVAV


- model is still failing, so we ask BuildingMOTIF for more feedback
- ask user for more inputs on some of the missing sensors

In [14]:
generated_templates = ctx.as_templates()
for t in generated_templates:
    print('-'*80)
    print(t.body.serialize())
    bindings = {}
    for param in t.parameters:
        ident = input(f"Give value for '{param}' in the above template: ")
        bindings[param] = BLDG[ident]
    model.add_graph(t.evaluate(bindings))

--------------------------------------------------------------------------------
@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:my_site/vav1> a ns1:RVAV,
        ns1:VAV ;
    ns1:hasPoint <urn:___param___#p1-489ea766-inlined> .

<urn:___param___#p1-489ea766-inlined> a ns1:Discharge_Air_Flow_Sensor .




Give value for 'p1-489ea766-inlined' in the above template:  dafs1


--------------------------------------------------------------------------------
@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:my_site/vav2> a ns1:RVAV,
        ns1:VAV ;
    ns1:hasPoint <urn:___param___#p2-575fc2e1-inlined> .

<urn:___param___#p2-575fc2e1-inlined> a ns1:Discharge_Air_Flow_Sensor .




Give value for 'p2-575fc2e1-inlined' in the above template:  dafs2


- BuildingMOTIF can add missing metadata automatically
    - if unambiguous (true for some constraints)
    - otherwise, attempt "autocomplete" from existing model

In [15]:
print(model.graph.serialize())

@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:my_site/> a <http://www.w3.org/2002/07/owl#Ontology> .

<urn:my_site/ahu> a <https://brickschema.org/schema/Brick#AHU> .

<urn:my_site/vav1> a <https://brickschema.org/schema/Brick#RVAV>,
        <https://brickschema.org/schema/Brick#VAV>,
        <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> ;
    ns1:hasPoint <urn:my_site/dafs1> .

<urn:my_site/vav2> a <https://brickschema.org/schema/Brick#RVAV>,
        <https://brickschema.org/schema/Brick#VAV>,
        <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> ;
    ns1:hasPoint <urn:my_site/dafs2> .

<urn:my_site/dafs1> a <https://brickschema.org/schema/Brick#Discharge_Air_Flow_Sensor> .

<urn:my_site/dafs2> a <https://brickschema.org/schema/Brick#Discharge_Air_Flow_Sensor> .




In [16]:
ctx = model.validate([manifest.get_shape_collection(), constraints.get_shape_collection(), g36.get_shape_collection(), brick.get_shape_collection()])
print(f"Is model valid yet? {ctx.valid}")

Is model valid yet? False


In [17]:
print(ctx.report.serialize())

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site/vav1> ;
            sh:resultMessage "Node <urn:my_site/vav1> does not conform to one or more shapes in <urn:ashrae/g36/4.2/vav-with-reheat/heating-signal1> , <urn:ashrae/g36/4.2/vav-with-reheat/heating-signal2>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> ;
            sh:value <urn:my_site/vav1> ],
        [ a sh:ValidationResult ;
            sh:focusNode <urn:my_site/vav1> ;
            sh:resultMessage "Node <urn:my_site/vav1> does not conform to one or more shapes in <urn:ashrae/g36/4.1/vav-cooling-only/occupancy-sensor1> , <urn:ashrae/g36/4.1/vav-cooling-only/occupancy-sensor2> , <urn:ashrae/g36/4.1/vav-coolin

 - we can also invoke templates ourselves, rather than having BuildingMOTIF suggest them
 - here using `fill()` as a debug trick to avoid having to give inputs

In [18]:
# use templates to finish it up!
vav_templ = g36.get_template_by_name("vav-with-reheat").inline_dependencies()
_, graph = vav_templ.evaluate({"name": BLDG["vav1"]}).fill(BLDG)
model.add_graph(graph)
_, graph = vav_templ.evaluate({"name": BLDG["vav2"]}).fill(BLDG)
model.add_graph(graph)

In [19]:
ctx = model.validate([manifest.get_shape_collection(),
                      constraints.get_shape_collection(),
                      g36.get_shape_collection(),
                      brick.get_shape_collection()])
print(f"Is model valid yet? {ctx.valid}")

Is model valid yet? True


In [20]:
print(model.graph.serialize())

@prefix ns1: <https://brickschema.org/schema/Brick#> .

<urn:my_site/> a <http://www.w3.org/2002/07/owl#Ontology> .

<urn:my_site/ahu> a <https://brickschema.org/schema/Brick#AHU> .

<urn:my_site/vav1> a <https://brickschema.org/schema/Brick#RVAV>,
        <https://brickschema.org/schema/Brick#VAV>,
        <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> ;
    ns1:feeds <urn:my_site/name-zone_2e78b113> ;
    ns1:hasPart <urn:my_site/name-dmp_e7ebe720>,
        <urn:my_site/rhc_f5208268> ;
    ns1:hasPoint <urn:my_site/dafs1>,
        <urn:my_site/name-dat_819d29db>,
        <urn:my_site/name-ztemp_23cc213e> .

<urn:my_site/vav2> a <https://brickschema.org/schema/Brick#RVAV>,
        <https://brickschema.org/schema/Brick#VAV>,
        <urn:ashrae/g36/4.2/vav-with-reheat/vav-with-reheat> ;
    ns1:feeds <urn:my_site/name-zone_1ae94b53> ;
    ns1:hasPart <urn:my_site/name-dmp_e33f0544>,
        <urn:my_site/rhc_11513b3b> ;
    ns1:hasPoint <urn:my_site/dafs2>,
        <urn:my_site/na

In [21]:
print(ctx.report.serialize())

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms true .


